**Загрузка данных через  API kaggle**

In [0]:
!pip install kaggle

In [0]:
import os
user = ''
key = ''
 
if '.kaggle' not in os.listdir('/root'):
    !mkdir ~/.kaggle
!touch /root/.kaggle/kaggle.json
!chmod 666 /root/.kaggle/kaggle.json
with open('/root/.kaggle/kaggle.json', 'w') as f:
    f.write('{"username":"%s","key":"%s"}' % (user, key))
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
!kaggle competitions download -c histopathologic-cancer-detection

In [0]:
!ls

In [0]:
!unzip train.zip -d train

In [0]:
!unzip test.zip -d test

In [0]:
!unzip train_labels.csv.zip

In [0]:
!unzip sample_submission.csv.zip

In [0]:
!ls

**Подготовка данных**

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [0]:
# Путь к датафрейму
train_labels = 'train_labels.csv'

In [0]:
df = pd.read_csv(train_labels)

#удаление нежелательных данных, обнаруженные другими пользователями kaggle
df = df[df['id'] != 'dd6dfed324f9fcb6f93f46f32fc800f2ec196be2']
df = df[df['id'] != '9369c7278ec8bcc6c880d99194de09fc2bd4efbe']

In [0]:
print(df['label'].value_counts(), 
      '\n\n', df.describe(), 
      '\n\n', df.head())

0    130907
1     89116
Name: label, dtype: int64 

                label
count  220023.000000
mean        0.405030
std         0.490899
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000 

                                          id  label
0  f38a6374c348f90b587e046aac6079959adf3835      0
1  c18f2d887b7ae4f6742ee445113fa1aef383ed77      1
2  755db6279dae599ebb4d39a9123cce439965282d      0
3  bc3f0c64fb968ff4a8bd33af6971ecae77c75e08      0
4  068aba587a4950175d04c680d38943fd488d6a9d      0


In [0]:
# Добавления к данным приставку '.tif' что бы датафрейм можно было использовать в flow_from_dataframe
df["id"]=df["id"].apply(lambda x : x +".tif")
df['label'] = df['label'].astype('str')
df.head()

,id,label
0,f38a6374c348f90b587e046aac6079959adf3835.tif,0
1,c18f2d887b7ae4f6742ee445113fa1aef383ed77.tif,1
2,755db6279dae599ebb4d39a9123cce439965282d.tif,0
3,bc3f0c64fb968ff4a8bd33af6971ecae77c75e08.tif,0
4,068aba587a4950175d04c680d38943fd488d6a9d.tif,0


**Инициализация генераторов Keras**

In [0]:
# кол-во снимков для валидации
val = .15

# Генератор данных
datagen= ImageDataGenerator(
    rescale=1./255,
    samplewise_std_normalization= True,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=90,
    zoom_range=0.2, 
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.05,
    channel_shift_range=0.1,
    validation_split=val)

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:353: UserWarning: This ImageDataGenerator specifies `samplewise_std_normalization`, which overrides setting of `samplewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


In [0]:
'''
# кол-во снимков для валидации
val = .1

# Генератор данных для второй сети
datagen= ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2
    ,height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    samplewise_std_normalization=True,
    samplewise_center=True,
    horizontal_flip=True,
    vertical_flip=True,
    channel_shift_range=0.1,
    validation_split=val)
'''

In [0]:
# Генератор данных для обучения

train_generator=datagen.flow_from_dataframe(
    dataframe=df,
    directory='train',
    x_col="id",
    y_col="label",
    subset="training",
    batch_size=128,
    shuffle=True,
    class_mode="binary",
    target_size=(96,96))


# Генератор валидационных данных

valid_generator=datagen.flow_from_dataframe(
    dataframe=df,
    directory='train',
    x_col="id",
    y_col="label",
    subset="validation",
    batch_size=128,
    shuffle=True,
    class_mode="binary",
    target_size=(96,96))


Found 187020 images belonging to 2 classes.
Found 33003 images belonging to 2 classes.


**Построение модели**

In [0]:
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.applications.xception import Xception
from keras.callbacks import ModelCheckpoint
from keras.layers import GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam,Adamax,Nadam

In [0]:
# Загрузка предтренировочной сети Xception

base_model = Xception(weights='imagenet', include_top=False, input_shape=(96,96,3))

Instructions for updating:
Colocations handled automatically by placer.
83689472/83683744 [==============================] - 1s 0us/step


In [0]:
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 47, 47, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 47, 47, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 47, 47, 32)   0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [0]:
#Проходимся циклом по архитектуре сети чтобы разморозить веса на нужном слое
base_model.trainable = True
#trainable = False
#for layer in base_model.layers:
#  if layer.name == 'conv2d_1':
#    trainable =True
#  layer.trainable = trainable

In [0]:
# Строительство архитектуры сети
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(1,activation='sigmoid'))

In [0]:
# Проверка количества обучаемых весов
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 3, 3, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 2049      
Total params: 20,863,529
Trainable params: 20,809,001
Non-trainable params: 54,528
_________________________________________________________________


In [0]:
# Точки сохранения во время обучения для лучших и последних параметров сети,ориентир L
best_w = ModelCheckpoint('best_xceptoion.h5',
                        monitor='val_loss',
                        verbose=0,
                        save_best_only=True,
                        save_weights_only=True,
                        mode='auto',
                        period=3)
last_w = ModelCheckpoint('last_xception.h5',
                        monitor='val_loss',
                        verbose=0,
                        save_best_only=False,
                        save_weights_only=True,
                        mode='auto',
                        period=1)

callbacks = [best_w,last_w]

In [0]:
'''
epochs: 4+2
lr:1e-04,0.002
'''

In [0]:
# Методы градиентного спуска
#adam = Adam(lr=2e-2, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
adamax = Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
#nadam = Nadam(lr=1e-2, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)

In [0]:
# Компиляция модели
model.compile(optimizer=adamax,
              loss='binary_crossentropy',
              metrics=['accuracy'])

#1e-04,0.002



**Score: 0.9667**

In [0]:
# Начало обучения
history = model.fit_generator(train_generator,
                              steps_per_epoch=train_generator.n//128, 
                              validation_data=valid_generator,
                              validation_steps=valid_generator.n//128,
                              epochs=2,
                              verbose=1,
                              callbacks=callbacks)

Epoch 1/2
1461/1461 [==============================] - 1770s 1s/step - loss: 0.1388 - acc: 0.9487 - val_loss: 0.1336 - val_acc: 0.9523
Epoch 2/2
1461/1461 [==============================] - 1814s 1s/step - loss: 0.1308 - acc: 0.9523 - val_loss: 0.1286 - val_acc: 0.9537


In [0]:
## Сохранение нейронной сети
import h5py
# сохранение архетектуры модели в JSON
model_json = model.to_json()
with open("xception.json", "w") as json_file:
    json_file.write(model_json)
    
# сохранение весов в HDF5
model.save_weights("one.h5")
print("Модель сохранена")

Модель сохранена


In [0]:
!ls

In [0]:
# Переинициализировать генераторы данных
from keras.models import model_from_json

## Загрузка и создания модели
import h5py
# загрузка архитектуры
json_file = open('xception.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
## Загрузка весов 
model.load_weights('one.h5')
print("Загружена предтренировочная сеть")

Загружена предтренировочная сеть


In [0]:
## Загрузка данных из файловой системы google colaborratory в локальную через его API
from google.colab import files
#files.upload()
#files.download('')

**Предсказания**

In [0]:
# путь к папке с тестовыми снимками
test_path = 'test'

In [0]:
df_test = pd.read_csv('sample_submission.csv')
df_test["id"]=df_test["id"].apply(lambda x : x +".tif")

In [0]:
# Малые изменения 
test_datagen = ImageDataGenerator(rescale=1./255,
                                 samplewise_std_normalization= True)


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:353: UserWarning: This ImageDataGenerator specifies `samplewise_std_normalization`, which overrides setting of `samplewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


In [0]:
## Измненения данных аналогичны тренировочным
# Генератор данных
test_datagen= ImageDataGenerator(
    rescale=1./255,
    samplewise_std_normalization= True,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=90,
    zoom_range=0.2, 
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.05,
    channel_shift_range=0.1)

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:353: UserWarning: This ImageDataGenerator specifies `samplewise_std_normalization`, which overrides setting of `samplewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


In [0]:
'''
#Third aug
test_datagen= ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2
    ,height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    samplewise_std_normalization=True,
    samplewise_center=True,
    horizontal_flip=True,
    vertical_flip=True,
    channel_shift_range=0.1)
    
'''

In [0]:
## Тестовый генератор не перемешиваеться, и classmode так же не передается в качестве аргумента.

test_generator = test_datagen.flow_from_dataframe(
    dataframe=df_test,
    directory=test_path,
    x_col="id",
    y_col=None,
    target_size=(96, 96),
    color_mode="rgb",
    batch_size=128,
    class_mode=None,
    shuffle=False)

Found 57458 images.


In [0]:
## Предсказания 
test_generator.reset()
pred=model.predict_generator(test_generator,steps=450,verbose=1).ravel()

450/450 [==============================] - 329s 732ms/step


**CSV Посылка**

**Предсказания тестового генератора находятся не в правильном порядке, поэтому его необходимо переставить в списке меток, прежде чем передать во фрейм данных отправки.**

In [0]:
results = dict(zip(test_generator.filenames, pred))

label = []
for i in range(len(df_test["id"])):
    label.append(results[df_test["id"][i]])
    
df_test["id"]=df_test["id"].apply(lambda x : x[:-4])


In [0]:
submission=pd.DataFrame({"id":df_test["id"],
                      "label":label})
submission.to_csv("jjj.csv",index=False)
submission.head()


,id,label
0,0b2ea2a822ad23fdb1b5dd26653da899fbd2c0d5,0.000390
1,95596b92e5066c5c52466c90b69ff089b39f2737,0.029846
2,248e6738860e2ebcf6258cdc1f32f299e0c76914,0.007561
3,2c35657e312966e9294eac6841726ff3a748febf,0.011245
4,145782eb7caa1c516acbe2eda34d9a3f31c41fd6,0.089992


In [0]:
from google.colab import files
#files.upload()
files.download('jjj.csv')